In [1]:
import mbuild as mb
import warnings 
warnings.filterwarnings('ignore')

In [2]:
butane = mb.load("CCCC", smiles=True)
butane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
box = mb.fill_box(compound = butane,
                 n_compounds=1, 
                 density=0.01)

In [4]:
box.visualize() #is this supposed to show the box around it?

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
import foyer


In [6]:
gaff = foyer.Forcefield(forcefield_files = "../repos/GAFF-foyer/gafffoyer/xml/gaff.xml")

FileNotFoundError: [Errno 2] No such file or directory: '../repos/GAFF-foyer/gafffoyer/xml/gaff.xml'

In [ ]:
box_typed = gaff.apply(box)

In [ ]:
box_typed.strip([a.atomic_number == 1 for a in box_typed])

In [ ]:
type(box_typed) 
#parmed structure contains all the info like bond distance and stickiness in a file that can be used to run a hoomd simulation

In [ ]:
type(box)

In [ ]:
for atom in box_typed.atoms:
    print(atom.sigma)
    print(atom.epsilon)

In [ ]:
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [ ]:
help(create_hoomd_forcefield)

In [ ]:
snap, forcefield, refs = create_hoomd_forcefield(box_typed, r_cut = 2.5, auto_scale=True)

In [ ]:
print(forcefield)

In [ ]:
import hoomd
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snap)

In [ ]:
integrator = hoomd.md.Integrator(dt=0.0001)
nvt = hoomd.md.methods.NVT(kT=1.5,filter=hoomd.filter.All(),tau=0.1)
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.5)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_dihedral.gsd",
    mode="wb"
)
sim.operations.writers.append(gsd_writer)

In [ ]:
sim.run(5e5)

In [ ]:
import gsd.hoomd

In [ ]:
traj = gsd.hoomd.open("traj_dihedral.gsd")

In [ ]:
len(traj)

In [ ]:
snapshot = traj[10]

In [ ]:
snapshot.particles.position

In [ ]:
from cmeutils.structure import dihedral_distribution

In [ ]:
dist = dihedral_distribution('traj_dihedral.gsd',A_name = "c3", B_name = "c3", C_name = "c3", D_name = "c3", histogram = True)